In [1]:
import os
import pandas as pd
from PIL import Image
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report


data_folder = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/data/Multi_Label_dataset/Images"
poster_filenames = os.listdir(data_folder)
metadata_filename = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/data/IMDb movies.csv"
metadata = pd.read_csv(metadata_filename)

# Prétraitement des données
poster_size = (128, 128)
num_classes = len(metadata["genre"].unique())
genre_list = list(metadata["genre"].unique())
X = np.empty((len(poster_filenames), poster_size[0], poster_size[1], 3), dtype=np.uint8)
y = np.zeros((len(poster_filenames), num_classes), dtype=np.uint8)
for i, filename in enumerate(poster_filenames):
    img = Image.open(os.path.join(data_folder, filename)).convert("RGB")
    img = img.resize(poster_size)
    X[i] = np.array(img)
    imdb_id = os.path.splitext(filename)[0]
    genre = metadata[metadata["imdb_title_id"] == imdb_id]["genre"]
    for g in genre:
        y[i, genre_list.index(g)] = 1


# Création du modèle CNN
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu", input_shape=(poster_size[0], poster_size[1], 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=num_classes, activation="softmax"))
model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

# Entraînement du modèle
model.fit(X, y, batch_size=32, epochs=10, validation_split=0.2)

# Évaluation du modèle
y_pred = model.predict(X)
predicted_genre_indices = np.argmax(y_pred, axis=1)
predicted_genres = [genre_list[i] for i in predicted_genre_indices]
true_genre_indices = np.argmax(y, axis=1)
true_genres = [genre_list[i] for i in true_genre_indices]
print("Classification Report:\n", classification_report(true_genres, predicted_genres))

# Prédiction sur un poster donné
poster_filename = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/outer.jpg"
img = Image.open(poster_filename).convert("RGB")
img = img.resize(poster_size)
X_pred = np.array(img).reshape(1, poster_size[0], poster_size[1], 3)
y_pred = model.predict(X_pred)
predicted_genre = genre_list[np.argmax(y_pred)]
print("Predicted genre:", predicted_genre)




ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import os
import pandas as pd
from PIL import Image
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

In [ ]:
data_folder = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/data/Multi_Label_dataset/Images"
poster_filenames = os.listdir(data_folder)
metadata_filename = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/data/IMDb movies.csv"
metadata = pd.read_csv(metadata_filename)

# Prétraitement des données
poster_size = (128, 128)
num_classes = len(metadata["genre"].unique())
genre_list = list(metadata["genre"].unique())
X = np.empty((len(poster_filenames), poster_size[0], poster_size[1], 3), dtype=np.uint8)
y = np.zeros((len(poster_filenames), num_classes), dtype=np.uint8)
for i, filename in enumerate(poster_filenames):
    img = Image.open(os.path.join(data_folder, filename)).convert("RGB")
    img = img.resize(poster_size)
    X[i] = np.array(img)
    imdb_id = os.path.splitext(filename)[0]
    genre = metadata[metadata["imdb_title_id"] == imdb_id]["genre"]
    for g in genre:
        y[i, genre_list.index(g)] = 1



In [ ]:
from tensorflow.keras import regularizers

# Création du modèle CNN avec régularisation L2
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu", kernel_regularizer=regularizers.l2(0.01), input_shape=(poster_size[0], poster_size[1], 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu", kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=128, activation="relu", kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(units=num_classes, activation="softmax", kernel_regularizer=regularizers.l2(0.01)))

# Compilation du modèle
model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])


In [ ]:
# Entraînement du modèle
model.fit(X, y, batch_size=32, epochs=10, validation_split=0.2)

In [ ]:
# Évaluation du modèle
y_pred = model.predict(X)
predicted_genre_indices = np.argmax(y_pred, axis=1)
predicted_genres = [genre_list[i] for i in predicted_genre_indices]
true_genre_indices = np.argmax(y, axis=1)
true_genres = [genre_list[i] for i in true_genre_indices]
print("Classification Report:\n", classification_report(true_genres, predicted_genres))

In [ ]:
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# Évaluation du modèle
y_pred = model.predict(X)
predicted_genre_indices = np.argmax(y_pred, axis=1)
predicted_genres = [genre_list[i] for i in predicted_genre_indices]
true_genre_indices = np.argmax(y, axis=1)
true_genres = [genre_list[i] for i in true_genre_indices]

# Rapport de classification
report = classification_report(true_genres, predicted_genres, output_dict=True)

# Création du graphique
precision = []
recall = []
f1_score = []
for genre in genre_list:
    if genre in report:
        precision.append(report[genre]['precision'])
        recall.append(report[genre]['recall'])
        f1_score.append(report[genre]['f1-score'])
    else:
        precision.append(0)
        recall.append(0)
        f1_score.append(0)

fig, ax = plt.subplots()
index = np.arange(len(genre_list))
bar_width = 0.3
opacity = 0.8

rects1 = ax.bar(index, precision, bar_width,
                alpha=opacity, color='b',
                label='Precision')

rects2 = ax.bar(index + bar_width, recall, bar_width,
                alpha=opacity, color='g',
                label='Recall')

rects3 = ax.bar(index + 2*bar_width, f1_score, bar_width,
                alpha=opacity, color='r',
                label='F1-score')

ax.set_xlabel('Genres')
ax.set_ylabel('Scores')
ax.set_title('Classification Report')
ax.set_xticks(index + bar_width)
ax.set_xticklabels(genre_list, rotation=90)
ax.legend()

fig.tight_layout()
plt.show()


In [ ]:
# Prédiction sur un poster donné
poster_filename = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/TheNun.jpg"
img = Image.open(poster_filename).convert("RGB")
img = img.resize(poster_size)
X_pred = np.array(img).reshape(1, poster_size[0], poster_size[1], 3)
y_pred = model.predict(X_pred)
predicted_genre = genre_list[np.argmax(y_pred)]
print("Predicted genre:", predicted_genre)



In [ ]:
report = classification_report(true_genres, predicted_genres, output_dict=True)
genre_precision = {}
for genre in genre_list:
    genre_precision[genre] = report[genre]['precision']
    
genre_with_highest_precision = max(genre_precision, key=genre_precision.get)
print("Genre with highest precision:", genre_with_highest_precision)


In [ ]:
genre_precision = {}
for genre in genre_list:
    genre_precision[genre] = report.get(genre, {'precision': 0})['precision']

genre_with_highest_precision = max(genre_precision, key=genre_precision.get)
print("Genre with highest precision:", genre_with_highest_precision)


In [ ]:
import os
import pandas as pd
from PIL import Image
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report



In [ ]:
data_folder = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/data/Multi_Label_dataset/Images"
poster_filenames = os.listdir(data_folder)
metadata_filename = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/data/IMDb movies.csv"
metadata = pd.read_csv(metadata_filename)



In [ ]:
# Prétraitement des données
poster_size = (128, 128, 1)  # Modification de la taille pour inclure l'image en niveaux de gris
num_classes = len(metadata["genre"].unique())
genre_list = list(metadata["genre"].unique())
X = np.empty((len(poster_filenames), poster_size[0], poster_size[1], poster_size[2]), dtype=np.uint8)  # Modification de la dimension
y = np.zeros((len(poster_filenames), num_classes), dtype=np.uint8)
for i, filename in enumerate(poster_filenames):
    img = Image.open(os.path.join(data_folder, filename)).convert("L")  # Modification de la conversion en niveaux de gris
    img = img.resize((poster_size[0], poster_size[1]))  # Redimensionnement
    X[i] = np.array(img).reshape(poster_size)  # Modification pour inclure la dimension supplémentaire
    imdb_id = os.path.splitext(filename)[0]
    genre = metadata[metadata["imdb_title_id"] == imdb_id]["genre"]
    for g in genre:
        y[i, genre_list.index(g)] = 1

In [ ]:
# Création du modèle CNN
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu", input_shape=(poster_size[0], poster_size[1], poster_size[2])))  # Modification de la valeur de input_shape
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=num_classes, activation="softmax"))
model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])



In [ ]:

# Entraînement du modèle
model.fit(X, y, batch_size=32, epochs=10, validation_split=0.2)

In [ ]:
# Évaluation du modèle
y_pred = model.predict(X)
predicted_genre_indices = np.argmax(y_pred, axis=1)
predicted_genres = [genre_list[i] for i in predicted_genre_indices]
true_genre_indices = np.argmax(y, axis=1)
true_genres = [genre_list[i] for i in true_genre_indices]
print("Classification Report:\n", classification_report(true_genres, predicted_genres))

In [ ]:



# Prédiction sur un poster donné
poster_filename = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/TheNun.jpg"
img = Image.open(poster_filename).convert("L")  # Conversion en niveaux de gris
img = img.resize((poster_size[0], poster_size[1]))  # Redimensionnement
X_pred = np.array(img).reshape(1, poster_size[0], poster_size[1], poster_size[2])  # Modification du nombre de canaux
y_pred = model.predict(X_pred)
predicted_genre_indices = np.argmax(y_pred, axis=1)
predicted_genres = [genre_list[i] for i in predicted_genre_indices]
predicted_genre = predicted_genres[0]
print("Predicted genre:", predicted_genre)



In [ ]:
from PIL import Image

def extraire_couleur_predominante(image_path):
    # Charger l'image
    image = Image.open(image_path)

    # Convertir l'image en mode RGB
    image = image.convert('RGB')

    # Obtenir les dimensions de l'image
    largeur, hauteur = image.size

    # Créer un histogramme des couleurs
    histogramme = image.histogram()

    # Trouver l'indice de la valeur maximale dans l'histogramme
    couleur_predominante_index = max(enumerate(histogramme), key=lambda x: x[1])[0]

    # Convertir l'indice en valeurs RGB
    couleur_predominante = (
        couleur_predominante_index // (256 * 256),  # Rouge
        (couleur_predominante_index // 256) % 256,  # Vert
        couleur_predominante_index % 256           # Bleu
    )

    return couleur_predominante

# Exemple d'utilisation
image_path = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/TheNun.jpg"
couleur = extraire_couleur_predominante(image_path)
print("Couleur prédominante : RGB", couleur)


In [ ]:
from skimage import io
from sklearn.cluster import KMeans

def extraire_couleur_predominante(image_path):
    # Charger l'image
    image = io.imread(image_path)

    # Redimensionner l'image si nécessaire
    # image = io.resize(image, (nouvelle_largeur, nouvelle_hauteur))

    # Aplatir l'image en un tableau 2D
    pixels = image.reshape(-1, 3)

    # Utiliser K-means pour regrouper les couleurs
    kmeans = KMeans(n_clusters=1)
    kmeans.fit(pixels)

    # Obtenir les centres des clusters
    couleurs_centrales = kmeans.cluster_centers_

    # Convertir les valeurs RGB en entiers
    couleur_predominante = tuple(map(int, couleurs_centrales[0]))

    return couleur_predominante

# Exemple d'utilisation
image_path = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/TheNun.jpg"
couleur = extraire_couleur_predominante(image_path)
print("Couleur prédominante : RGB", couleur)


In [ ]:
import os
import csv
from PIL import Image
from skimage import io
from sklearn.cluster import KMeans

def extraire_couleur_predominante(image_path):
     # Charger l'image
    image = io.imread(image_path)

    # Redimensionner l'image si nécessaire
    # image = io.resize(image, (nouvelle_largeur, nouvelle_hauteur))

    # Aplatir l'image en un tableau 2D
    pixels = image.reshape(-1, 3)

    # Utiliser K-means pour regrouper les couleurs
    kmeans = KMeans(n_clusters=1)
    kmeans.fit(pixels)

    # Obtenir les centres des clusters
    couleurs_centrales = kmeans.cluster_centers_

    # Convertir les valeurs RGB en entiers
    couleur_predominante = tuple(map(int, couleurs_centrales[0]))

    return couleur_predominante

# Chemin du dossier contenant les images
dossier_images = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/data/Multi_Label_dataset/Images"

# Chemin du fichier CSV de sortie
chemin_csv = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/csv"

# Liste pour stocker les données des images
donnees_images = []

# Parcourir les fichiers du dossier
for nom_fichier in os.listdir(dossier_images):
    chemin_image = os.path.join(dossier_images, nom_fichier)

    # Vérifier si le fichier est une image
    if os.path.isfile(chemin_image) and nom_fichier.lower().endswith(('.png', '.jpg', '.jpeg')):
        couleur_predominante = extraire_couleur_predominante(chemin_image)
        donnees_images.append((nom_fichier, couleur_predominante))

# Écrire les données dans le fichier CSV
with open(chemin_csv, 'w', newline='') as fichier_csv:
    writer = csv.writer(fichier_csv)
    writer.writerow(['Nom du fichier', 'Couleur prédominante (R,G,B)'])
    for donnee_image in donnees_images:
        writer.writerow(donnee_image)

print("Fichier CSV créé avec succès !")


In [ ]:
import os
import csv
from skimage import io
from sklearn.cluster import KMeans

def extraire_couleur_predominante(image_path):
    # Charger l'image
    image = io.imread(image_path)

    # Aplatir l'image en un tableau 2D
    pixels = image.reshape(-1, 3)

    # Utiliser K-means pour regrouper les couleurs
    kmeans = KMeans(n_clusters=1)
    kmeans.fit(pixels)

    # Obtenir les centres des clusters
    couleurs_centrales = kmeans.cluster_centers_

    # Convertir les valeurs RGB en entiers
    couleur_predominante = tuple(map(int, couleurs_centrales[0]))

    return couleur_predominante

# Chemin du dossier contenant les images
dossier_images = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/data/Multi_Label_dataset/Images"

# Chemin du fichier CSV de sortie
chemin_csv = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/csv"

# Liste pour stocker les données des images
donnees_images = []

# Parcourir récursivement les sous-dossiers et fichiers
for dossier_racine, sous_dossiers, fichiers in os.walk(dossier_images):
    for nom_fichier in fichiers:
        chemin_image = os.path.join(dossier_racine, nom_fichier)

        # Vérifier si le fichier est une image
        if nom_fichier.lower().endswith(('.png', '.jpg', '.jpeg')):
            couleur_predominante = extraire_couleur_predominante(chemin_image)
            donnees_images.append((chemin_image, couleur_predominante))

# Écrire les données dans le fichier CSV
with open(chemin_csv, 'w', newline='') as fichier_csv:
    writer = csv.writer(fichier_csv)
    writer.writerow(['Chemin de l\'image', 'Couleur prédominante (R,G,B)'])
    for donnee_image in donnees_images:
        writer.writerow(donnee_image)

print("Fichier CSV créé avec succès !")


In [ ]:
import cv2
from pytesseract import image_to_string

def detecter_titre(image_path):
    # Charger l'image avec OpenCV
    image = cv2.imread(image_path)

    # Convertir l'image en niveaux de gris pour faciliter la détection des caractères
    image_gris = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Utiliser pytesseract-mac pour effectuer l'OCR sur l'image en niveaux de gris
    texte = image_to_string(image_gris)

    # Extraire les coordonnées de la zone de texte détectée
    boite_texte = image_to_string(image_gris, boxes=True)
    boite_texte = boite_texte.splitlines()

    # Calculer la position verticale moyenne de la boîte de texte détectée
    positions_verticales = [int(line.split(' ')[1]) for line in boite_texte]
    position_moyenne = sum(positions_verticales) / len(positions_verticales)

    # Déterminer la position du titre dans l'image
    hauteur_image = image.shape[0]
    position_relative = position_moyenne / hauteur_image

    # Déterminer si la position est en haut, au milieu ou en bas
    seuil_haut = 0.3
    seuil_bas = 0.7

    if position_relative < seuil_haut:
        position = "haut"
    elif position_relative < seuil_bas:
        position = "milieu"
    else:
        position = "bas"

    return texte, position


# Exemple d'utilisation
image_path = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/TheNun.jpg"
titre, position_titre = detecter_titre(image_path)
print("Titre:", titre)
print("Position du titre:", position_titre)


TypeError: image_to_string() got an unexpected keyword argument 'boxes'

In [ ]:
import cv2
import pyocr
from PIL import Image

def detecter_titre(image_path):
    # Charger l'image avec OpenCV
    image = cv2.imread(image_path)

    # Convertir l'image en niveaux de gris pour faciliter la détection des caractères
    image_gris = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Initialiser l'outil OCR avec PyOCR
    outils_ocr = pyocr.get_available_tools()
    if len(outils_ocr) == 0:
        raise Exception("Aucun outil OCR disponible")
    ocr_tool = outils_ocr[0]

    # Utiliser l'outil OCR pour effectuer l'OCR sur l'image en niveaux de gris
    texte = ocr_tool.image_to_string(Image.fromarray(image_gris))

    # ... effectuer d'autres opérations de traitement du texte ...

    return texte


# Exemple d'utilisation
image_path = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/TheNun.jpg"
titre = detecter_titre(image_path)
print("Titre:", titre)


Titre: 


In [ ]:
import cv2
import pytesseract

def detecter_position_titre(image_path):
    # Charger l'image avec OpenCV
    image = cv2.imread(image_path)

    # Convertir l'image en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Appliquer un traitement d'image pour améliorer la lisibilité du texte (par exemple, seuillage adaptatif)
    processed = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

    # Utiliser Tesseract pour détecter les contours de texte
    contours, _ = cv2.findContours(processed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Trouver le contour le plus grand (qui correspond probablement au titre)
    contour_titre = max(contours, key=cv2.contourArea)

    # Calculer le rectangle englobant le contour du titre
    x, y, w, h = cv2.boundingRect(contour_titre)

    # Déterminer la position du titre en analysant la position verticale du rectangle englobant
    hauteur, largeur = image.shape[:2]
    position = ""
    position_verticale = y / hauteur
    if position_verticale < 0.3:
        position = "en haut"
    elif position_verticale > 0.7:
        position = "en bas"
    else:
        position = "au milieu"

    return position

# Chemin vers l'image PNG contenant le poster de film
chemin_image = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/vik.jpeg"

# Appeler la fonction pour détecter la position du titre
position_titre = detecter_position_titre(chemin_image)

# Afficher le résultat
print("Le titre se situe:", position_titre)


Le titre se situe: en haut


In [ ]:
import cv2
import pytesseract

def detecter_position_titre(image_path):
    # Charger l'image avec OpenCV
    image = cv2.imread(image_path)

    # Convertir l'image en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Appliquer un traitement d'image pour améliorer la lisibilité du texte (par exemple, seuillage adaptatif)
    processed = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

    # Utiliser Tesseract pour détecter les contours de texte
    contours, _ = cv2.findContours(processed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Trier les contours par taille décroissante
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    # Sélectionner les deux contours les plus grands (probablement le titre et un autre texte)
    premier_contour = contours[0]
    deuxieme_contour = contours[1]

    # Récupérer les coordonnées des rectangles englobants pour les deux contours
    x1, y1, w1, h1 = cv2.boundingRect(premier_contour)
    x2, y2, w2, h2 = cv2.boundingRect(deuxieme_contour)

    # Calculer les positions verticales des deux rectangles englobants
    hauteur, largeur = image.shape[:2]
    position_titre = y1 / hauteur
    position_autre = y2 / hauteur

    # Déterminer la position du titre en comparant les positions verticales
    position = ""
    if position_titre < position_autre:
        position = "en haut"
    else:
        position = "en bas"

    return position

# Chemin vers l'image PNG contenant le poster de film
chemin_image = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/TheNun.jpg"

# Appeler la fonction pour détecter la position du titre
position_titre = detecter_position_titre(chemin_image)

# Afficher le résultat
print("Le titre se situe:", position_titre)


Le titre se situe: en bas


In [ ]:
import cv2
import pytesseract

def detecter_position_titre(image_path):
    # Charger l'image avec OpenCV
    image = cv2.imread(image_path)

    # Convertir l'image en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Appliquer un traitement d'image pour améliorer la lisibilité du texte (par exemple, seuillage adaptatif)
    processed = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

    # Utiliser Tesseract pour détecter les contours de texte
    contours, _ = cv2.findContours(processed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Trier les contours par taille décroissante
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    # Sélectionner les deux contours les plus grands (probablement le titre et un autre texte)
    premier_contour = contours[0]
    deuxieme_contour = contours[1]

    # Récupérer les coordonnées des rectangles englobants pour les deux contours
    x1, y1, w1, h1 = cv2.boundingRect(premier_contour)
    x2, y2, w2, h2 = cv2.boundingRect(deuxieme_contour)

    # Détecter les textes dans les rectangles englobants
    titre = pytesseract.image_to_string(processed[y1:y1+h1, x1:x1+w1], lang='fra')
    autre_texte = pytesseract.image_to_string(processed[y2:y2+h2, x2:x2+w2], lang='fra')

    # Calculer les positions verticales des deux rectangles englobants
    hauteur, largeur = image.shape[:2]
    position_titre = y1 / hauteur
    position_autre = y2 / hauteur

    # Déterminer la position du titre en comparant les positions verticales
    position = ""
    if position_titre < position_autre:
        position = "en haut"
    else:
        position = "en bas"

    return position, titre, autre_texte

# Chemin vers l'image PNG contenant le poster de film
chemin_image = r"/Users/dimitriraymond/Documents/Epitech/Semestre 2/DAT/mer.jpeg"

# Appeler la fonction pour détecter la position du titre et récupérer les textes
position_titre, titre, autre_texte = detecter_position_titre(chemin_image)

# Afficher le résultat
print("Le titre se situe:", position_titre)
print("Titre:", titre)
print("Autre texte:", autre_texte)


TesseractError: (1, 'Error opening data file /opt/homebrew/share/tessdata/fra.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'fra\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')